In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from fuzzywuzzy import process,fuzz

import warnings
warnings.filterwarnings('ignore', category = UserWarning, module ='openpyxl')

D:\Python\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# reading dataset
df = pd.read_excel("D://Capstone_Project_ParthBhavnani//Assignments//Project_8//Datasets//dataset_01.xlsx")

In [3]:
df.head(5)

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [4]:
# Checking for Null Values
df.isna().sum() 

Internal ID                    0
Q1: GOING OUT?               110
Q2: GENDER                    41
Q3: AGE                       84
Q4: COUNTRY                   64
                            ... 
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
Length: 120, dtype: int64

In [5]:
# see quick info of numeric values
df.describe()

,Internal ID,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
count,2.460000e+03,85.0,1362.0,99.0,67.0
mean,9.028016e+07,1.0,1.0,1.0,1.0
std,7.435532e+03,0.0,0.0,0.0,0.0
min,9.025877e+07,1.0,1.0,1.0,1.0
25%,9.027520e+07,1.0,1.0,1.0,1.0
50%,9.027780e+07,1.0,1.0,1.0,1.0
75%,9.028242e+07,1.0,1.0,1.0,1.0
max,9.031480e+07,1.0,1.0,1.0,1.0


In [6]:
# see quick info of category values 
df.describe(include = object)

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),...,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,"Click Coordinates (x, y)"
count,2350,2419,2376,2396,2360,1713,1726,1788,1502,1468,...,1703,1713,1755,911,717,386,1716,9,1735,1605
unique,2,4,106,128,511,3,3,3,3,3,...,3,3,3,867,685,385,2,1,2,952
top,No,Male,40,USA,California,JOY,DESPAIR,JOY,DESPAIR,MEH,...,DESPAIR,DESPAIR,JOY,Almond Joy,Trump,nom nom nom!,White and gold,dress (https://survey.ubc.ca/media/assets/user...,Friday,"(77, 24)"
freq,2038,1467,92,699,132,873,1089,1559,793,857,...,1455,1289,1105,10,7,2,1081,9,1091,9


# Data - Cleaning

In [7]:
# show columns that hase more 50 % nan value

for i in range(len(df.isnull().sum())):
    if df.isna().sum()[i] > 1229 :
        print(f"Column {df.isna().sum().index[i]} contains {df.isna().sum()[i]} nan value\n")

Column Q7: JOY OTHER contains 1549 nan value

Column Q8: DESPAIR OTHER contains 1743 nan value

Column Q9: OTHER COMMENTS contains 2074 nan value

Column Unnamed: 113 contains 2451 nan value

Column Q12: MEDIA [Daily Dish] contains 2375 nan value

Column Q12: MEDIA [ESPN] contains 2361 nan value

Column Q12: MEDIA [Yahoo] contains 2393 nan value



In [8]:
# drop columns we could not use & which contain more than 50 % nan value

df.drop(columns = ["Q8: DESPAIR OTHER", "Q9: OTHER COMMENTS", "Unnamed: 113", "Q7: JOY OTHER"], inplace = True)

In [9]:
# handle columns name

# 1- create dictionary key is old name and value is new name
ColNames = {    "Q1: GOING OUT?": "Going out",
                "Q2: GENDER": "Gender",
                "Q3: AGE": "Age",
                "Q4: COUNTRY": "Country",
                "Q5: STATE, PROVINCE, COUNTY, ETC": "State/Province",
                "Q10: DRESS" : "Dress", 
                "Q12: MEDIA [Science]":"Science",
                "Q11: DAY": "Day", 
                "Q12: MEDIA [Daily Dish]":"Daily Dish",
                "Q12: MEDIA [ESPN]":"ESPN", 
                "Q12: MEDIA [Yahoo]":"Yahoo"}

# 2- rename columns
df.rename(columns = ColNames, inplace= True)

In [10]:
# show unique values
df["Going out"].unique()

array([nan, 'No', 'Yes'], dtype=object)

In [11]:
# see number of missing values 
df["Going out"].isna().sum()

110

In [12]:
# fill nan value with maybe (yse,no)
df["Going out"].fillna("May_Be",inplace = True)

In [13]:
# show unique values
df["Gender"].unique()

array([nan, 'Male', 'Female', "I'd rather not say", 'Other'], dtype=object)

In [14]:
# fill missing values 
df["Gender"].fillna("OTHER", inplace = True)

In [15]:
# replace Other with I'd rather not say
df["Gender"].replace("I'd rather not say", "OTHER", inplace = True)

In [16]:
# convert column data type to numiric and replace non-digits to NAN value
df['Age'] = pd.to_numeric(df["Age"], errors = "coerce")

In [17]:
# Calculate mean and standard deviation
mean_age = df["Age"].mean()
std_age = df["Age"].std()

# Set the threshold for identifying outliers (e.g., 3 standard deviations from the mean)
threshold = 3

# Calculate the upper and lower bounds
Upper = mean_age + threshold * std_age
Lower = mean_age - threshold * std_age

# Replace outliers with NaN
df.loc[(df["Age"] > Upper) | (df["Age"] < Lower), "Age"] = np.nan

# Replace NaN values with median
median_age = df["Age"].median()
df["Age"].fillna(median_age, inplace=True)

In [18]:
# 1- convert all value to Upper case
df["Country"] = df["Country"].str.upper()

# 2- remove extra spaces 
df["Country"] = df["Country"].str.strip()

In [19]:
df["Country"].fillna("OTHER", inplace = True)

In [20]:
# 1- convert all value to Upper case
df["State/Province"] = df["State/Province"].str.upper()

# 2- remove extra spaces 
df["State/Province"] = df["State/Province"].str.strip()

In [21]:
df["State/Province"].fillna("OTHER", inplace = True)

In [22]:
countries = pd.Series(['USA', 'UNITED STATES OF AMERICA','AMERICA','UNITED KINGDOM ENGLAND', 'HONG KONG', 'CHINA', 'SWEDEN', 'SPAIN','IRELAND', 'TAIWAN', 'SINGAPORE', 'NETHERLANDS','INDONESIA', 'SOUTH KOREA', 'SWITZERLAND', 'DENMARK', 'SCOTLAND', 'CANADA', 'ICELAND', 'SOUTH AFRICA',
                       'JAPAN', 'GREECE', 'COSTA RICA', 'GERMANY', 'FINLAND', 'FRANCE', 'MEXICO'])

for country in countries:
    matches = process.extract(country, df['Country'], limit = df.shape[0])
    for potential_match in matches:
        if potential_match[1] >= 80:
            df.loc[df['Country'] == potential_match[0], 'Country'] = country

In [23]:
# show unique values again
df["Country"].unique()

array(['OTHER', 'USA', 'CANADA', 'MURICA', 'AMERICA', 'UK', 'UAE',
       'UNITED KINGDOM ENGLAND', 'MEXICO', 'U.S.A.', 'FRANCE', 'FINLAND',
       'NORTH CAROLINA', 'NETHERLANDS', 'GERMANY', 'EUROPE', 'EARTH',
       'U.K.', 'COSTA RICA', 'CASCADIA', 'AUSTRALIA', 'INSANITY LATELY',
       'GREECE', 'USA? HARD TO TELL ANYMORE..', 'PITTSBURGH', 'CANAE',
       'NEW YORK', 'TRUMPISTAN', 'ICELAND', 'SOUTH KOREA', 'CALIFORNIA',
       'JAPAN', 'SOUTH AFRICA', 'SCOTLAND', 'DENMARK', 'SWITZERLAND',
       'UD', 'NEW JERSEY', 'INDONESIA', 'ENDLAND', 'ATLANTIS', 'MURRIKA',
       'ALASKA', 'SOVIET CANUCKISTAN', 'SINGAPORE', 'CHINA', 'TAIWAN',
       'HONG KONG', 'SPAIN', 'SWEDEN', 'NARNIA', 'U S A',
       'SUBSCRIBE TO DM4UZ3 ON YOUTUBE', "I DON'T KNOW ANYMORE",
       'FEAR AND LOATHING'], dtype=object)

In [24]:
CountryName = {'MURICA':'USA', 'UNITED STATES OF AMERICA':'USA','AMERICA':'USA', 'U.S.A.':'USA','NORTH CAROLINA':'USA', 'USA? HARD TO TELL ANYMORE..':'USA', 'PITTSBURGH':'USA', 'NEW YORK':'USA', 'TRUMPISTAN':'USA',
          'CALIFORNIA':'USA', 'UD':'USA', 'NEW JERSEY':'USA', 'ENDLAND':'UK', 'SOVIET CANUCKISTAN':'CANADA', 'U S A':'USA', 'UNITED KINGDOM ENGLAND':'UK', 'U.K.':'UK', 'CASCADIA':'USA', 'SUBSCRIBE TO DM4UZ3 ON YOUTUBE' : 'OTHER',
       "I DON'T KNOW ANYMORE" : 'OTHER', 'FEAR AND LOATHING' : 'OTHER'}
df['Country'] = df['Country'].replace(CountryName)

In [25]:
# show unique values again
df["Country"].unique()

array(['OTHER', 'USA', 'CANADA', 'UK', 'UAE', 'MEXICO', 'FRANCE',
       'FINLAND', 'NETHERLANDS', 'GERMANY', 'EUROPE', 'EARTH',
       'COSTA RICA', 'AUSTRALIA', 'INSANITY LATELY', 'GREECE', 'CANAE',
       'ICELAND', 'SOUTH KOREA', 'JAPAN', 'SOUTH AFRICA', 'SCOTLAND',
       'DENMARK', 'SWITZERLAND', 'INDONESIA', 'ATLANTIS', 'MURRIKA',
       'ALASKA', 'SINGAPORE', 'CHINA', 'TAIWAN', 'HONG KONG', 'SPAIN',
       'SWEDEN', 'NARNIA'], dtype=object)

Cleaning dress column

In [26]:
# see number of missing values 
df["Dress"].isna().sum()

744

In [27]:
# fill nan value
df["Dress"].fillna("OTHER", inplace = True)

Cleaning day column

In [28]:
# see number of missing values 
df["Day"].isna().sum()

725

In [29]:
# fill nan value 
df["Day"].fillna("OTHER", inplace = True)  

Create new DataFrame

In [30]:
# 1- create list of Question columns 
Questions = [candy for candy in df.columns if 'Q6' in str(candy)]

# 2- create dataframe call candy
Candies = pd.DataFrame(df[Questions])

In [31]:
Candies.sample(5)

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
963,MEH,DESPAIR,JOY,NaN,NaN,NaN,NaN,DESPAIR,DESPAIR,JOY,...,JOY,JOY,MEH,JOY,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY
275,MEH,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY
1455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,...,JOY,JOY,DESPAIR,MEH,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR
350,JOY,MEH,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,...,JOY,JOY,JOY,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,JOY


In [32]:
# Total nan value in each column
Candies.isna().sum()

Q6 | 100 Grand Bar                                                                        747
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)    734
Q6 | Any full-sized candy bar                                                             672
Q6 | Black Jacks                                                                          958
Q6 | Bonkers (the candy)                                                                  992
                                                                                         ... 
Q6 | Vicodin                                                                              789
Q6 | Whatchamacallit Bars                                                                 823
Q6 | White Bread                                                                          757
Q6 | Whole Wheat anything                                                                 747
Q6 | York Peppermint Patties                                

In [33]:
Q6_columns = df.columns[df.columns.str.contains('Q6')]
df[Q6_columns] = df[Q6_columns].fillna('MEH')
Q6=df[Q6_columns].copy()
Q6.head()

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
0,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,...,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH
1,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR
2,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,...,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH,MEH
3,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR
4,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY


In [34]:
Q6 = Q6.replace(['JOY','DESPAIR','MEH'],[1, -1, 0])

In [35]:
candy_rate_list = []
for column in Q6.columns:
    candy_rate = Q6[column].sum() / Q6[column].nunique()
    candy_rate_list.append(candy_rate)
candy_df = pd.DataFrame({
    'candy rate': candy_rate_list
})

In [36]:
# Create Candy dataset
questions = [candy for candy in df.columns if 'Q6' in str(candy)]
candies_df = df[questions].copy()
candies_df.fillna("Other", inplace=True)

# Rename Candy dataset columns
candy_col_mapping = {}
for col in candies_df.columns:
    candy_col_mapping[col] = col.strip("Q6 | ")
candies_df.rename(columns=candy_col_mapping, inplace=True)

# Update column names in the original dataset
df.drop(columns=questions, inplace=True)
df = pd.concat([df, candies_df], axis=1)


In [37]:
# Export DataFrame to CSV file
df.to_csv('Cleaned_Dataset_1.csv', index=False)